In [29]:
import ipyrad as ip
import ipyrad.analysis as ipa
import ipyparallel as ipp
import pandas as pd
import toyplot
import glob
import gzip

## Ptychadena (Stephane)
## * http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0190440#sec022

## Peromyscus (Munshi-South)

## Chinese sea bass (SRP094869)
## Population Genomics Reveals Genetic Divergence and Adaptive Differentiation of Chinese Sea Bass (Lateolabrax maculatus)
## * https://link.springer.com/article/10.1007/s10126-017-9786-0#Sec2

In [ ]:
%%bash
conda install -c eaton-lab toytree
conda install -c bioconda sra-tools entrez-direct

In [ ]:
%%bash
## Fetch the elephant reference genome
mkdir ref
wget ftp://ftp.broadinstitute.org/pub/assemblies/mammals/elephant/loxAfr3/assembly_supers.fasta.gz -o ref/assembly_supers.fasta.gz

In [ ]:
%%bash

## Data from "Single-nucleotide polymorphism discovery and panel characterization in the African forest elephant"
## http://onlinelibrary.wiley.com/doi/10.1002/ece3.3854/full?wol1URL=/doi/10.1002/ece3.3854/full&regionCode=US-NY&identityKey=34b23ec9-4666-4c37-8470-8448e64d6167
ipyrad --download SRP126637 raws/

In [39]:
## R1 and R2 were concatenated in the SRA data files so we have to pull them apart.
## This is probably not the most efficient way to do this, but it works.
raws = glob.glob("raws/*")
print(raws)
for r in raws:
    name = r.split("/")[1].split("_")[0]
    lines = gzip.open(r).readlines()
    nlines = len(lines)/2
    print("{} {}".format(name, nlines))
    with gzip.open("raws/{}_R1_.fastq.gz".format(name), 'wb') as r1:
        r1.write("".join(lines[:nlines]))
    with gzip.open("raws/{}_R2_.fastq.gz".format(name), 'wb') as r2:
        r2.write("".join(lines[nlines:]))


['raws/LOC0310_SRR6371511.fastq.gz', 'raws/LOC0088_SRR6371521.fastq.gz', 'raws/LOC0311_SRR6371510.fastq.gz', 'raws/LOC0051_SRR6371516.fastq.gz', 'raws/LOC0037_SRR6371514.fastq.gz', 'raws/LOC0038_SRR6371513.fastq.gz', 'raws/LOC0040_SRR6371512.fastq.gz', 'raws/LOC0041_SRR6371519.fastq.gz', 'raws/LOC0050_SRR6371517.fastq.gz', 'raws/LOC0309_SRR6371508.fastq.gz', 'raws/LOC0151_SRR6371505.fastq.gz', 'raws/LOC0127_SRR6371502.fastq.gz', 'raws/LOC0279_SRR6371509.fastq.gz', 'raws/LOC0274_SRR6371506.fastq.gz', 'raws/LOC0049_SRR6371518.fastq.gz', 'raws/LOC0035_SRR6371515.fastq.gz', 'raws/LOC0121_SRR6371520.fastq.gz', 'raws/LOC0263_SRR6371507.fastq.gz', 'raws/LOC0122_SRR6371503.fastq.gz']
LOC0310 535328
LOC0088 2532764
LOC0311 2381720
LOC0051 2267296
LOC0037 4639748
LOC0038 4352988
LOC0040 1516120
LOC0041 2733056
LOC0050 3633896
LOC0309 2004280
LOC0151 2005558
LOC0127 482392
LOC0279 5789736
LOC0274 3165976
LOC0049 2942484
LOC0035 1812120
LOC0121 450136
LOC0263 5038456
LOC0122 2266816


```
Fetching project data...
           Run    spots mates      ScientificName SampleName
0   SRR6371502   241196     0  Loxodonta cyclotis    LOC0127
1   SRR6371503  1133408     0  Loxodonta cyclotis    LOC0122
2   SRR6371505  1002779     0  Loxodonta cyclotis    LOC0151
3   SRR6371506  1582988     0  Loxodonta cyclotis    LOC0274
4   SRR6371507  2519228     0  Loxodonta cyclotis    LOC0263
5   SRR6371508  1002140     0  Loxodonta cyclotis    LOC0309
6   SRR6371509  2894868     0  Loxodonta cyclotis    LOC0279
7   SRR6371510  1190860     0  Loxodonta cyclotis    LOC0311
8   SRR6371511   267664     0  Loxodonta cyclotis    LOC0310
9   SRR6371512   758060     0  Loxodonta cyclotis    LOC0040
10  SRR6371513  2176494     0  Loxodonta cyclotis    LOC0038
11  SRR6371514  2319874     0  Loxodonta cyclotis    LOC0037
12  SRR6371515   906060     0  Loxodonta cyclotis    LOC0035
13  SRR6371516  1133648     0  Loxodonta cyclotis    LOC0051
14  SRR6371517  1816948     0  Loxodonta cyclotis    LOC0050
15  SRR6371518  1471242     0  Loxodonta cyclotis    LOC0049
16  SRR6371519  1366528     0  Loxodonta cyclotis    LOC0041
17  SRR6371520   225068     0  Loxodonta cyclotis    LOC0121
18  SRR6371521  1266382     0  Loxodonta cyclotis    LOC0088
```

In [37]:
df = pd.DataFrame([x.split("\t") for x in """SRR6371521	SAMN08167496	LOC0088	WG011	178	95	SRX3466825	LOC0088_a	Lope
SRR6371520	SAMN08167497	LOC0121	WG012	31	16	SRX3466826	LOC0121_a	Minkebe
SRR6371519	SAMN08167492	LOC0041	WG005	192	102	SRX3466827	LOC0041_a	Waka
SRR6371518	SAMN08167493	LOC0049	WG008	207	110	SRX3466828	LOC0049_a	Ivindo
SRR6371517	SAMN08167494	LOC0050	WG009	256	135	SRX3466829	LOC0050_b	Ivindo
SRR6371516	SAMN08167495	LOC0051	WG010	160	85	SRX3466830	LOC0051_a	Ivindo
SRR6371515	SAMN08167488	LOC0035	WG001	127	69	SRX3466831	LOC0035_a	Minkebe
SRR6371514	SAMN08167489	LOC0037	WG002	327	175	SRX3466832	LOC0037_a	Lope
SRR6371513	SAMN08167490	LOC0038	WG003	307	164	SRX3466833	LOC0038_a	Lope
SRR6371512	SAMN08167491	LOC0040	WG004	106	56	SRX3466834	LOC0040_a	Wonga Wongue
SRR6371511	SAMN08167506	LOC0310	WG023	37	20	SRX3466835	LOC0310_a	Moukalaba Doudou
SRR6371510	SAMN08167507	LOC0311	WG024	168	89	SRX3466836	LOC0311_a	Monts de Cristal
SRR6371509	SAMN08167504	LOC0279	WG020	408	215	SRX3466837	LOC0279_b	South Mulundu
SRR6371508	SAMN08167505	LOC0309	WG022	141	75	SRX3466838	LOC0309_a	Mayumba
SRR6371507	SAMN08167502	LOC0263	WG018	355	188	SRX3466839	LOC0263_a	Wonga Wongue
SRR6371506	SAMN08167503	LOC0274	WG019	223	117	SRX3466840	LOC0274_a	Loango
SRR6371505	SAMN08167500	LOC0151	WG015	141	71	SRX3466841	LOC0151_a	Moukalaba Doudou
SRR6371503	SAMN08167498	LOC0122	WG013	159	85	SRX3466843	LOC0122_a	Minkebe
SRR6371502	SAMN08167499	LOC0127	WG014	34	18	SRX3466844	LOC0127_a	Moukalaba Doudou""".split("\n")])

## Just get the sequence identifier, the sample name, and the sample location
df = df.iloc[:, [0,7,8]]


             0          7                 8
0   SRR6371521  LOC0088_a              Lope
1   SRR6371520  LOC0121_a           Minkebe
2   SRR6371519  LOC0041_a              Waka
3   SRR6371518  LOC0049_a            Ivindo
4   SRR6371517  LOC0050_b            Ivindo
5   SRR6371516  LOC0051_a            Ivindo
6   SRR6371515  LOC0035_a           Minkebe
7   SRR6371514  LOC0037_a              Lope
8   SRR6371513  LOC0038_a              Lope
9   SRR6371512  LOC0040_a      Wonga Wongue
10  SRR6371511  LOC0310_a  Moukalaba Doudou
11  SRR6371510  LOC0311_a  Monts de Cristal
12  SRR6371509  LOC0279_b     South Mulundu
13  SRR6371508  LOC0309_a           Mayumba
14  SRR6371507  LOC0263_a      Wonga Wongue
15  SRR6371506  LOC0274_a            Loango
16  SRR6371505  LOC0151_a  Moukalaba Doudou
17  SRR6371503  LOC0122_a           Minkebe
18  SRR6371502  LOC0127_a  Moukalaba Doudou


Launch the cluster by running this command on the command line:

    ipcluster start --n=19 --daemonize

In [5]:
## connect to cluster and check if all the engines are running
ipyclient = ipp.Client()
ipyclient.ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [19]:
data = ip.Assembly("Loxodonta")

New Assembly: Loxodonta


In [37]:
## set parameters
data.set_params("project_dir", "ddrad-denovo")
data.set_params("sorted_fastq_path", "raws/*_.fastq")
data.set_params("datatype", "pairddrad")
data.set_params("restriction_overhang", ("TGCAG", "CATGC"))
data.set_params("clust_threshold", "0.90")
data.set_params("filter_adapters", "2")
data.set_params("max_Hs_consens", (5, 5))
data.set_params("trim_loci", (0, 0, 0, 0))
data.set_params("output_formats", "*")

## see/print all parameters
data.get_params()
data.write_params(force=True)

0   assembly_name               Loxodonta                                    
1   project_dir                 ./ddrad-denovo                               
2   raw_fastq_path                                                           
3   barcodes_path                                                            
4   sorted_fastq_path           ./raws/*_.fastq                              
5   assembly_method             denovo                                       
6   reference_sequence                                                       
7   datatype                    pairddrad                                    
8   restriction_overhang        ('TGCAG', 'CATGC')                           
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         33                                           
11  mindepth_statistical        6                                            
12  mindepth_majrule            6                               

In [40]:
data.run("12")

Assembly: Loxodonta
    Skipping: 19 Samples already found in Assembly Loxodonta.
    (can overwrite with force argument)    
[####################] 100%  processing reads      | 0:00:02 | s2 | 
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in step2; see ipyrad_log.txt
  found an error in ste

In [41]:
## access the stats of the assembly (so far) from the .stats attribute
data.stats

,state,reads_raw,reads_passed_filter
LOC0035_SRR6371515,2,906060,905955
LOC0037_SRR6371514,2,2319874,2319590
LOC0038_SRR6371513,2,2176494,2176182
LOC0040_SRR6371512,2,758060,757942
LOC0041_SRR6371519,2,1366528,1366341
LOC0049_SRR6371518,2,1471242,1471081
LOC0050_SRR6371517,2,1816948,1816737
LOC0051_SRR6371516,2,1133648,1133530
LOC0088_SRR6371521,2,1266382,1266181
LOC0121_SRR6371520,2,225068,225031


In [42]:
## run steps 3-6 of the assembly
data.run("34567")

Assembly: Loxodonta
[####################] 100%  dereplicating         | 0:00:22 | s3 | 
[####################] 100%  clustering            | 0:06:02 | s3 | 
[####################] 100%  building clusters     | 0:00:21 | s3 | 
[####################] 100%  chunking              | 0:00:03 | s3 | 
[####################] 100%  aligning              | 0:20:57 | s3 | 
[####################] 100%  concatenating         | 0:00:13 | s3 | 
[####################] 100%  inferring [H, E]      | 0:01:48 | s4 | 
[####################] 100%  calculating depths    | 0:00:04 | s5 | 
[####################] 100%  chunking clusters     | 0:00:04 | s5 | 
[####################] 100%  consens calling       | 0:03:03 | s5 | 
[####################] 100%  concat/shuffle input  | 0:00:04 | s6 | 
[####################] 100%  clustering across     | 0:02:13 | s6 | 
[####################] 100%  building clusters     | 0:00:03 | s6 | 
[####################] 100%  aligning clusters     | 0:01:07 | s6 | 
[#############

In [7]:
kvalues = [2, 3, 4, 5, 6]
s = ipa.structure(
    name="quick",
    workdir="./analysis-structure",
    data="./ddrad-denovo/Loxodonta_outfiles/Loxodonta.ustr",
    )

## set main params (use much larger values in a real analysis)
s.mainparams.burnin = 1000
s.mainparams.numreps = 5000

## submit N replicates of each test to run on parallel client
for kpop in kvalues:
    s.run(kpop=kpop, nreps=4, ipyclient=ipyclient)

## wait for parallel jobs to finish
ipyclient.wait()

submitted 4 structure jobs [quick-K-2]
submitted 4 structure jobs [quick-K-3]
submitted 4 structure jobs [quick-K-4]
submitted 4 structure jobs [quick-K-5]
submitted 4 structure jobs [quick-K-6]


True

In [8]:
## return the evanno table (deltaK) for best K 
etable = s.get_evanno_table(kvalues)
etable

,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,4,0.000,-5.503e+05,5.116e+05,0.000,0.000e+00
3,4,3.163,-3.206e+05,3.185e+05,229741.725,1.007e+06
4,4,0.455,-1.098e+06,6.932e+05,-777577.375,3.152e+05
5,4,1.264,-1.560e+06,6.283e+05,-462336.825,7.943e+05
6,4,0.000,-1.229e+06,1.059e+06,331935.450,0.000e+00


In [17]:
## set some clumpp params
s.clumppparams.m = 3               ## use largegreedy algorithm
s.clumppparams.greedy_option = 2   ## test nrepeat possible orders
s.clumppparams.repeats = 10000     ## number of repeats
s.clumppparams
## run clumpp for each value of K
#tables = s.get_clumpp_table(kvalues, quiet=True)
print(tables)
#table = tables[4].sort_values(by=[0, 1, 2, 3])
table = tables[2].sort_values(by=[0, 1])
toyplot.bars(
    table,
    width=500, 
    height=200,
    title=[[i] for i in table.index.tolist()],
    xshow=False,
);
print(table)

{2:                             0          1
LOC0035_SRR6371515  2.510e-01  7.490e-01
LOC0037_SRR6371514  4.995e-01  5.005e-01
LOC0038_SRR6371513  4.972e-01  5.027e-01
LOC0040_SRR6371512  1.000e-03  9.990e-01
LOC0041_SRR6371519  5.835e-01  4.165e-01
LOC0049_SRR6371518  9.995e-01  5.000e-04
LOC0050_SRR6371517  1.000e+00  0.000e+00
LOC0051_SRR6371516  9.988e-01  1.300e-03
LOC0088_SRR6371521  3.583e-01  6.418e-01
LOC0121_SRR6371520  2.000e-03  9.980e-01
LOC0122_SRR6371503  2.700e-03  9.972e-01
LOC0127_SRR6371502  1.512e-01  8.487e-01
LOC0151_SRR6371505  2.515e-01  7.485e-01
LOC0263_SRR6371507  8.000e-04  9.992e-01
LOC0274_SRR6371506  2.507e-01  7.492e-01
LOC0279_SRR6371509  4.995e-01  5.005e-01
LOC0309_SRR6371508  3.872e-01  6.128e-01
LOC0310_SRR6371511  2.500e-01  7.500e-01
LOC0311_SRR6371510  4.992e-01  5.008e-01, 3:                             0          1          2
LOC0035_SRR6371515  1.500e-03  9.800e-02  9.005e-01
LOC0037_SRR6371514  3.000e-04  9.995e-01  3.000e-04
LOC0038_SRR63715

LOC0263_SRR6371507 LOC0040_SRR6371512 LOC0121_SRR6371520 LOC0122_SRR6371503 LOC0127_SRR6371502 LOC0310_SRR6371511 LOC0274_SRR6371506 LOC0035_SRR6371515 LOC0151_SRR6371505 LOC0088_SRR6371521 LOC0309_SRR6371508 LOC0038_SRR6371513 LOC0311_SRR6371510 LOC0037_SRR6371514 LOC0279_SRR6371509 LOC0041_SRR6371519 LOC0051_SRR6371516 LOC0049_SRR6371518 LOC0050_SRR6371517 LOC0263_SRR6371507 LOC0040_SRR6371512 LOC0121_SRR6371520 LOC0122_SRR6371503 LOC0127_SRR6371502 LOC0310_SRR6371511 LOC0274_SRR6371506 LOC0035_SRR6371515 LOC0151_SRR6371505 LOC0088_SRR6371521 LOC0309_SRR6371508 LOC0038_SRR6371513 LOC0311_SRR6371510 LOC0037_SRR6371514 LOC0279_SRR6371509 LOC0041_SRR6371519 LOC0051_SRR6371516 LOC0049_SRR6371518 LOC0050_SRR6371517 0.0 0.5 1.0